In [10]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt


In [11]:
def articulationVowels(audioA, audioE, audioI, audioO, audioU, path_base):
    size_frameS=0.04
    size_stepS=0.02
    F1a, F2a=formants(audioA, path_base, size_frameS, size_stepS)
    F1e, F2e=formants(audioE, path_base, size_frameS, size_stepS)
    F1i, F2i=formants(audioI, path_base, size_frameS, size_stepS)
    F1o, F2o=formants(audioO, path_base, size_frameS, size_stepS)
    F1u, F2u=formants(audioU, path_base, size_frameS, size_stepS)
    mF1a=F1a.mean(0)
    mF1e=F1e.mean(0)
    mF1i=F1i.mean(0)
    mF1o=F1o.mean(0)
    mF1u=F1u.mean(0)
    mF2a=F2a.mean(0)
    mF2e=F2e.mean(0)
    mF2i=F2i.mean(0)
    mF2o=F2o.mean(0)
    mF2u=F2u.mean(0)
    VSA=(mF1i*np.abs(mF2a-mF2u)+mF1a*np.abs(mF2u-mF2i)+mF1u*np.abs(mF2i-mF2a))/2
    VAI=(mF2i+mF1a)/(mF2u+mF2a+mF1i+mF1u)
    ps1=mF1a*mF2o-mF1o*mF2a
    ps2=mF1o*mF2u-mF1u*mF2o
    ps3=mF1u*mF2i-mF1i*mF2u
    ps4=mF1i*mF2e-mF1e*mF2i
    ps5=mF1e*mF2a-mF1a*mF2e
    VPA=np.abs(ps1+ps2+ps3+ps4+ps5)/2.0
#    plt.scatter(F1a, F2a, c='b', alpha=0.5, marker='o')
#    plt.scatter(F1i, F2i, c='r', alpha=0.5, marker='x')
#    plt.scatter(F1u, F2u, c='k', alpha=0.5, marker='+')
#    plt.plot([mF1a, mF1i, mF1u, mF1a], [mF2a, mF2i, mF2u, mF2a], linewidth=3)
#    plt.show()
    
#    plt.scatter(F1a, F2a, c='b', alpha=0.5, marker='o')
#    plt.scatter(F1e, F2e, c='r', alpha=0.5, marker='o')
#    plt.scatter(F1i, F2i, c='r', alpha=0.5, marker='x')
#    plt.scatter(F1o, F2o, c='g', alpha=0.5, marker='o')
#    plt.scatter(F1u, F2u, c='k', alpha=0.5, marker='+')
#    plt.plot([mF1a, mF1e, mF1i, mF1u, mF1o, mF1a], [mF2a, mF2e, mF2i, mF2u, mF2o, mF2a], linewidth=3)
#    plt.show()    
    
    return F1a, F1e, F1i, F1o, F1u, F2a, F2e, F2i, F2o, F2u, VSA, VAI, VPA

    
def formants(audio, path_base, size_frame, time_shift):
    os.system(path_base+'../../Toolkits/praat '+ path_base+'FormantsPraat.praat '+audio+' '+path_base+'tempF12.txt 5 5500 '+' '+str((size_frame))+' '+str((time_shift))) #formant extraction praat
    [F1, F2]=decodeFormants(path_base+'tempF12.txt')
    return F1, F2    
    
    
def decodeFormants(fileTxt):
    fid=open(fileTxt)
    datam=fid.read()
    end_line1=multi_find(datam, '\n')
    F1=[]
    F2=[]
    ji=10
    while (ji<len(end_line1)-1):
        line1=datam[end_line1[ji]+1:end_line1[ji+1]]
        cond=(line1=='3' or line1=='4' or line1=='5')
        if (cond): 
            F1.append(float(datam[end_line1[ji+1]+1:end_line1[ji+2]]))
            F2.append(float(datam[end_line1[ji+3]+1:end_line1[ji+4]]))
        ji=ji+1
    F1=np.asarray(F1)
    F2=np.asarray(F2)
    return F1, F2    


def multi_find(s, r):
    s_len = len(s)
    r_len = len(r)
    _complete = []
    if s_len < r_len:
        n = -1
    else:
        for i in range(s_len):
            # search for r in s until not enough characters are left
            if s[i:i + r_len] == r:
                _complete.append(i)
            else:
                i = i + 1
    return(_complete)


In [12]:
path_base='C:/Users/elmon/Dropbox/Parkinson/PDTool/src/artVowels/'
file_features='C:/Users\elmon/Dropbox/Parkinson/PDTool/src/artVowels/referenceHCmale.txt'
path_audio='C:/Users/elmon/Desktop/pcgitavowelsHC/male/'

hf =os.listdir(path_audio)
hf.sort()
Features=[]
for j in range(len(hf)):    
    try:
        audioA=path_audio+hf[j]
        audioE=path_audio+hf[j+3]
        audioI=path_audio+hf[j+6]
        audioO=path_audio+hf[j+9]
        audioU=path_audio+hf[j+12]
        ID=int(hf[j][0:3])
        task=hf[j][4:6]
        if ID>50:
            if task=='a1' or task=='a2' or task=='a3':
                F1a, F1e, F1i, F1o, F1u, F2a, F2e, F2i, F2o, F2u, VSA, VAI, VPA=articulationVowels(audioA, audioE, audioI, audioO, audioU, path_base)
                feat=[VSA, 1.0/VAI, VPA, np.mean(F1a), np.mean(F2a), np.mean(F1e), np.mean(F2e), np.mean(F1i), np.mean(F2i), np.mean(F1o), np.mean(F2o), np.mean(F1u), np.mean(F2u)]
                Features.append(feat)
                print(hf[j],hf[j+3], hf[j+6], hf[j+9], hf[j+12], feat)
        if ID==100:
            break
    except:
        break
np.savetxt(file_features, Features, fmt="%.2f")


052_a1_PCGITA.wav 052_e1_PCGITA.wav 052_i1_PCGITA.wav 052_o1_PCGITA.wav 052_u1_PCGITA.wav [465186.00978607847, 1.128824941137385, 162177.11230152624, 652.07511358596457, 1142.6942544659651, 472.83322432543366, 1660.9178418107924, 319.43652865620817, 2083.0828533725321, 459.07721030166789, 870.9288021480312, 375.27020102767187, 1250.1135469035291]
052_a2_PCGITA.wav 052_e2_PCGITA.wav 052_i2_PCGITA.wav 052_o2_PCGITA.wav 052_u2_PCGITA.wav [374181.04333451344, 1.3481163410320665, 24572.64105395996, 604.33240195116264, 1175.5176198926911, 424.93015090968169, 1728.90395896934, 311.47420548179849, 2042.7659669983093, 497.80400316233806, 1313.9367751061695, 422.03955589025651, 1659.5651862353675]
052_a3_PCGITA.wav 052_e3_PCGITA.wav 052_i3_PCGITA.wav 052_o3_PCGITA.wav 052_u3_PCGITA.wav [431473.60261199146, 1.2156978467849699, 75218.964608951821, 537.01589275155823, 1046.8997730708361, 430.39643443244643, 1716.4093772693857, 316.44333153738046, 2059.7961330742905, 468.74935660031099, 922.48452271